In [2]:
# Notebook ssetup
import os
import numpy as np
import pandas as pd
import re

# directories
clean = 'Clean_Data/'
raw = 'Foundation/Raw Data/'
root = '/root/'

# Zip Code Ground Truth

In [3]:
# read in GROUND TRUTH
file_path = root + raw + 'Other/' + 'Zip_Codes_by_Court_District_GROUND_TRUTH.csv'
official_zip_code_df = pd.read_csv(file_path)
official_zip_codes = official_zip_code_df['ZIP CODE']
official_zip_codes[:3]

0    91901.0
1    91902.0
2    91903.0
Name: ZIP CODE, dtype: float64

# Vehicle DataPrep

In [4]:
file_path = root + 'Foundation/Raw Data/Other/Vehicle Population.xlsx'
veh_df = pd.read_excel(file_path,
                       sheet_name='ZIP')

# print shape
print(f'Shape {veh_df.shape}\n', '--'*20)

veh_df.head(3)

Shape (175291, 4)
 ----------------------------------------


,Data Year,Fuel Type,ZIP,Number of Vehicles
0,2010,Bio Diesel,90010,3
1,2010,Bio Diesel,90019,1
2,2010,Bio Diesel,90047,1


In [5]:
veh_df.rename(columns={'Data Year':'Year',
                        'Fuel Type':'Fuel',
                        'Number of Vehicles':'#_vehicles'},
              inplace=True)

#### Remove unwanted zip codes

In [6]:
veh_df[veh_df['#_vehicles'] == 0]

,Year,Fuel,ZIP,#_vehicles


In [7]:
veh_df.shape

(175291, 4)

In [8]:
veh_df = veh_df[veh_df['ZIP'].isin(official_zip_codes)].copy()
veh_df.shape

(12744, 4)

# Melt Dataframe

In [9]:
years = sorted(set(veh_df['Year']))

dfs = []
for year in years:
    print(f'Processesing {year}', '\n')

    year_df = veh_df.loc[veh_df['Year'] == year]

    zip_codes = set(year_df['ZIP'])

    for zip_code in zip_codes:

        mutate_df = year_df.loc[year_df['ZIP'] == zip_code, ['Fuel', '#_vehicles']]

        indexes = list(mutate_df.T.iloc[0,:])

        mutate_df.index = pd.Index(indexes,name='Fuel')

        mutate_df = mutate_df.T.reset_index(drop=True)

        mutate_df['Year'] = year
        mutate_df['ZIP'] = zip_code

        dfs.append(pd.DataFrame(mutate_df.iloc[1,:]).T)

final_df = pd.concat(dfs)
final_df.head(3)

Processesing 2010 

Processesing 2011 

Processesing 2012 

Processesing 2013 

Processesing 2014 

Processesing 2015 

Processesing 2016 

Processesing 2017 

Processesing 2018 

Processesing 2019 

Processesing 2020 



,Diesel,Flex Fuel,Gasoline,Gasoline Hybrid,Year,ZIP,Natural Gas,Bio Diesel,Propane,Electric,PHEV,Hydrogen
1,7,7,293,8,2010,92160,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,4,NaN,2010,92161,NaN,NaN,NaN,NaN,NaN,NaN
1,5,4,184,2,2010,92162,NaN,NaN,NaN,NaN,NaN,NaN


Index is labeld as "FUEL".  This will be omitted when DataFrame is written to CSV/Parquet.

In [10]:
final_df = final_df.fillna(0)
final_df.head(3)

,Diesel,Flex Fuel,Gasoline,Gasoline Hybrid,Year,ZIP,Natural Gas,Bio Diesel,Propane,Electric,PHEV,Hydrogen
1,7,7,293,8,2010,92160,0,0,0,0,0,0
1,0,0,4,0,2010,92161,0,0,0,0,0,0
1,5,4,184,2,2010,92162,0,0,0,0,0,0


#### Fix Hydrogen column name
Hydrogen column name has an extra space at the end!

In [11]:
final_df = final_df.rename(columns={'Hydrogen ':'Hydrogen'})
final_df.columns

Index(['Diesel', 'Flex Fuel', 'Gasoline', 'Gasoline Hybrid', 'Year', 'ZIP',
       'Natural Gas', 'Bio Diesel', 'Propane', 'Electric', 'PHEV', 'Hydrogen'],
      dtype='object')

## Interpolate for dates missing
Utilizing Linear Padding

In [ ]:
final_df.loc[final_df['ZIP'] == 92090].sort_values('Year')

In [12]:
dfs = []
zip_check = []
years_needed_list = []
for zip_ in zip_codes:
    interpolate_df = final_df.loc[final_df['ZIP'] == zip_]

    # gather the years from df
    years = set(interpolate_df['Year'])
    # set requirement for years
    needed_years = set(list(np.arange(2010,2022)))

    # get the difference between the two
    years_needed = list(needed_years - years)

    # --- create rows for years needed ---
    # gather the years from df
    years = set(interpolate_df['Year'])
    # set requirement for years
    needed_years = set(list(np.arange(2010,2022)))

    # get the difference between the two
    years_needed = list(needed_years - years)
    
    # check if zip code is missing any crucial years (2010-2020)
    if pd.Series(years_needed).isin(np.arange(2011,2021))[0]:
        zip_check.append(zip_)
        years_needed_list.append(sorted(years_needed))
    
    # create rows for years needed

    for year_needed in years_needed:
        next_row_dict = {
            'Diesel': [np.nan],
            'Flex Fuel': [np.nan],
            'Gasoline': [np.nan],
            'Gasoline Hybrid': [np.nan],
            'Year': [year_needed],
            'ZIP': [zip_],
            'Natural Gas' : [np.nan],
            'Bio Diesel': [np.nan],
            'Propane': [np.nan],
            'Electric': [np.nan],
            'PHEV': [np.nan],
            'Hydrogen': [np.nan]
        }
        next_row_df = pd.DataFrame.from_dict(next_row_dict)
        interpolate_df = pd.concat([interpolate_df, next_row_df])
    
    interpolate_df = interpolate_df.reset_index(drop=True)
    # interpolate for new added years
    interpolate_df = interpolate_df.interpolate(method='linear', axis=0)
    dfs.append(interpolate_df)

final_df = pd.concat(dfs)
final_df.head(3)

,Diesel,Flex Fuel,Gasoline,Gasoline Hybrid,Year,ZIP,Natural Gas,Bio Diesel,Propane,Electric,PHEV,Hydrogen
0,7.0,7.0,293.0,8.0,2010,92160,0.0,0.0,0.0,0.0,0.0,0.0
1,8.0,9.0,305.0,7.0,2011,92160,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,12.0,330.0,10.0,2012,92160,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
zip_check

[92187, 91947, 91987, 92090]

In [14]:
years_needed_list

[[2010, 2015, 2016, 2019, 2021],
 [2019, 2021],
 [2011, 2015, 2021],
 [2012, 2013, 2015, 2016, 2018, 2021]]

In [15]:
final_df.loc[final_df['ZIP'] == 92090].sort_values('Year')

,Diesel,Flex Fuel,Gasoline,Gasoline Hybrid,Year,ZIP,Natural Gas,Bio Diesel,Propane,Electric,PHEV,Hydrogen
0,0.0,0.0,6.0,0.0,2010,92090,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,0.0,2011,92090,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,3.0,0.0,2012,92090,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,3.0,0.0,2013,92090,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,2014,92090,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,3.0,0.0,2015,92090,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,3.0,0.0,2016,92090,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,2017,92090,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,3.0,0.0,2018,92090,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,2019,92090,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
final_df = final_df.reset_index(drop=True)
final_df.head(3)

,Diesel,Flex Fuel,Gasoline,Gasoline Hybrid,Year,ZIP,Natural Gas,Bio Diesel,Propane,Electric,PHEV,Hydrogen
0,7.0,7.0,293.0,8.0,2010,92160,0.0,0.0,0.0,0.0,0.0,0.0
1,8.0,9.0,305.0,7.0,2011,92160,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,12.0,330.0,10.0,2012,92160,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
vehicle_columns = ['Bio Diesel',
                   'Diesel',
                   'Electric',
                   'Flex Fuel',
                   'Gasoline',
                   'Gasoline Hybrid',
                   'Hydrogen',
                   'Natural Gas',
                   'PHEV',
                   'Propane']

final_df['total_cars'] = final_df.loc[:,vehicle_columns].sum(axis=1)
final_df.head(3)

,Diesel,Flex Fuel,Gasoline,Gasoline Hybrid,Year,ZIP,Natural Gas,Bio Diesel,Propane,Electric,PHEV,Hydrogen,total_cars
0,7.0,7.0,293.0,8.0,2010,92160,0.0,0.0,0.0,0.0,0.0,0.0,315.0
1,8.0,9.0,305.0,7.0,2011,92160,0.0,0.0,0.0,0.0,0.0,0.0,329.0
2,12.0,12.0,330.0,10.0,2012,92160,0.0,0.0,0.0,0.0,1.0,0.0,365.0


# Write DF to CSV & Parquet

In [ ]:
clean_data_dir = root + clean

final_df.to_parquet(clean_data_dir + 'vehicle_registration.parquet.gzip',
                                   compression='gzip',
                                   index=False)

In [27]:

clean_data_dir = root + clean

final_df.to_csv(clean_data_dir +'vehicle_registration.csv', index=False)

## Upload Files to S3

In [29]:
import boto3
import sagemaker

# Sagemaker session
sess = sagemaker.Session()

# bucket = "my-bucket"
bucket = sess.default_bucket()

file_name = 'vehicle_registration.csv'
file_path = 'Clean_Data/' + file_name

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(clean_data_dir + file_name, bucket, file_path)

In [ ]:
import boto3
import sagemaker

# Sagemaker session
sess = sagemaker.Session()

# bucket = "my-bucket"
bucket = sess.default_bucket()

file_name = 'vehicle_registration.parquet.gzip'
file_path = 'Clean_Data/' + file_name

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(clean_data_dir + file_name, bucket, file_path)

In [ ]:
print('Vehicle Data is READY TO ROCK!!!!')